In [1]:
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')

# Check and handling variables

In [2]:
df = pd.read_excel('./250328_survey.xlsx')
df.drop('번호', axis=1, inplace=True)
variables = [ # BS5_1(1개월 내 금연계획 여부 제외)
    'FEV1', 'FEV1FVC', 'age', 'sex', 'occp', 'Edu', 'BS3_1', 'BS6_3', 'BS6_2_1',
    'BD1', 'EC1_1', '금연지지가족친구여부', 'marri_1', '손자손녀동거여부', '건강문해력',
    'HE_PFThs', 'HE_cough1', 'HE_sput1', '가래양상', 'BH9_11', 'DI1_dg', 'DE1_dg',
    'DC6_dg', 'BP1', 'BP5', 'D_1_1', 'Total_slp_wk', 'EQ_5D', 'BS5_1', '자기효능감',
    'N_DUSUAL', 'BE3_31', 'BE5_1', 'LQ4_05', 'BH1_1', 'BO2_1', 'BO1_1', '질환유병기간',
    'DI1_pt', 'DI1_2', 'DE1_pt', 'DE1_3', '심한금단증상경험여부', '전문가의금연권유'
]
df.columns = variables
df.drop('BS5_1', axis=1, inplace=True)

In [3]:
c_name = ['가래양상', 'BO1_1', '전문가의금연권유', 'EC1_1', '질환유병기간', 'Total_slp_wk']
for c in c_name:
    print(f'{c} --> nan: {df[c].isna().sum()}')
    print(df[c].unique())

가래양상 --> nan: 82
[2 nan 1 3 0 4 '3, 4' '2, 4']
BO1_1 --> nan: 1
[1 2 3 4 '2,3' nan]
전문가의금연권유 --> nan: 1
[4 1 '1,2' 3 2 '1, 2, 3' '1, 2' '1, 3 ' '1, 3' nan '1,2,3' '1,3' '2,3']
EC1_1 --> nan: 0
[ 1  2 22  3]
질환유병기간 --> nan: 0
[  11    1   14    7    5    8   21   34   22    6   64    3    9   47
   20   10    4    2   15   24    0   30   32   44   16   17   53   18
   23   13   54   41   12   63   19   27 2024   28   25   52   31]
Total_slp_wk --> nan: 0
[7 6 4 8 5 1.5 10 3 9 15 12 2 '4~5' '5~6' 11 1 3.5 7.5 5.5 6.5 13 9.5]


In [4]:
df['가래양상'].replace(['3, 4', '2, 4'], [3, 2], inplace=True)
df['BO1_1'].replace('2,3', 2, inplace=True)
df['전문가의금연권유'].replace(['1,2', '1, 2, 3', '1, 2', '1, 3 ', '1, 3', '1,2,3', '1,3', '2,3'], 1, inplace=True)
df['EC1_1'].replace(22, np.nan, inplace=True)
df.loc[df['질환유병기간']==2024, '질환유병기간'] = 1
df['Total_slp_wk'].replace({'4~5':4.5, '5~6':5.5}, inplace=True)
c_name = ['가래양상', 'BO1_1', '전문가의금연권유', 'EC1_1', '질환유병기간', 'Total_slp_wk']
for c in c_name:
    print(f'{c} --> nan: {df[c].isna().sum()}')
    print(df[c].unique())

가래양상 --> nan: 82
[ 2. nan  1.  3.  0.  4.]
BO1_1 --> nan: 1
[ 1.  2.  3.  4. nan]
전문가의금연권유 --> nan: 1
[ 4.  1.  3.  2. nan]
EC1_1 --> nan: 1
[ 1.  2. nan  3.]
질환유병기간 --> nan: 0
[11  1 14  7  5  8 21 34 22  6 64  3  9 47 20 10  4  2 15 24  0 30 32 44
 16 17 53 18 23 13 54 41 12 63 19 27 28 25 52 31]
Total_slp_wk --> nan: 0
[ 7.   6.   4.   8.   5.   1.5 10.   3.   9.  15.  12.   2.   4.5  5.5
 11.   1.   3.5  7.5  6.5 13.   9.5]


### sex

In [5]:
# 1: 남자 2: 여자
df['sex'].unique()

array([1, 2])

In [6]:
# (변경) 0: 남자 1: 여자
df['sex'] = df['sex']-1
df['sex'].unique()

array([0, 1])

### occp

In [7]:
# 1: 있다 2: 없다 0: ??
df['occp'].unique()

array([1, 2, 0])

In [8]:
# (변경) 기존0 --> NaN, 0: 없다 1: 있다
df['occp'].replace({0:np.nan, 2:0}, inplace=True)
df['occp'].unique()

array([ 1.,  0., nan])

### Edu

In [9]:
# 1: 초졸이하 2: 중졸 3: 고졸 4: 대졸이상
df['Edu'].unique()

array([1, 3, 2, 4])

In [10]:
# (변경) 0: 초졸이하, 1: 나머지 (중졸~대졸이상)
df['Edu'].replace(
    {1:0, 2:1, 3:1, 4:1}, inplace=True)
df['Edu'].unique()

array([0, 1])

### BS3_1 (흡연여부, Target)

In [11]:
# 1: 매일피움 2: 가끔피움 3: 현재 안피움(과거피움) 4: 평생 비흡연
df['BS3_1'].unique()

array([4, 3, 1, 2])

In [12]:
# (변경) 0: 비흡연 (3~4) 1: 흡연 (1~2)
df['BS3_1'].replace({2:1, 3:0, 4:0}, inplace=True)
df['BS3_1'].unique()

array([0, 1])

### EC1_1

In [13]:
# 1: 유 2: 무 3, 22: ?? (NaN으로 변경됨)
df['EC1_1'].unique()

array([ 1.,  2., nan,  3.])

In [14]:
# (변경) 0: 무 1: 유 기존 22 --> NaN
df['EC1_1'].replace({2:0, 3:np.nan}, inplace=True)
df['EC1_1'].unique()

array([ 1.,  0., nan])

In [15]:
# 엑셀에서 확인 결과 (occp와 EC1_1은 동일한 의미)
# occp의 error value(1건) --> EC1_1에서 경제활동x
# EC1_1의 error value(2건) --> occp의 직업x
df['occp'].fillna(0, inplace=True)
df['EC1_1'].fillna(0, inplace=True)
# 350건 중 320건이 동일, 30건이 상이함...
# 경제활동상태 유지, occp(직업 유무)는 제거 (취소)
# df.drop('occp', axis=1, inplace=True)

### 금연지지가족친구여부

In [16]:
# 1: 유 2: 무 0: ??
df['금연지지가족친구여부'].unique()

array([2, 1, 0])

In [17]:
# (변경) 기존 0 --> NaN, 0: 무 1: 유
df['금연지지가족친구여부'].replace({2:0, 0:np.nan}, inplace=True)
df['금연지지가족친구여부'].unique()

array([ 0.,  1., nan])

### marri_1

In [18]:
# 1: 기혼 2: 미혼 3: ??
df['marri_1'].unique()

array([1, 3, 2])

In [19]:
# (변경) 기존 3-->NaN, 0: 미혼 1:기혼
df['marri_1'].replace({3:np.nan, 2:0}, inplace=True)
df['marri_1'].unique()

array([ 1., nan,  0.])

### 손자손녀동거여부

In [20]:
# 1: 예 2: 아니오 0: ??
df['손자손녀동거여부'].unique()

array([2, 1, 0])

In [21]:
# (변경) 0: 아니오 1: 예 기존0 --> NaN
df['손자손녀동거여부'].replace({2:0, 0:np.nan}, inplace=True)
df['손자손녀동거여부'].unique()

array([ 0.,  1., nan])

### HE_PFThs

In [22]:
# 0: 아니오 1: 예 2: ???? NaN
df['HE_PFThs'].unique()

array([ 0.,  1.,  2., nan])

In [23]:
# (변경) 2 --> NaN
df['HE_PFThs'].replace(2, np.nan, inplace=True)
df['HE_PFThs'].unique()

array([ 0.,  1., nan])

### 가래양상

In [24]:
# 0,1,2,3,4: ??? --> 0은 없음, 1234는 가래유형으로 추정
df['가래양상'].unique()

array([ 2., nan,  1.,  3.,  0.,  4.])

### BH9_11

In [25]:
# 1: 예 2: 아니오
df['BH9_11'].unique()

array([1, 2])

In [26]:
# (변경) 0:아니오 1: 예
df['BH9_11'].replace(2, 0, inplace=True)
df['BH9_11'].unique()

array([1, 0])

### BP1

In [27]:
# 0: ?? ,1: 대단히 많이 느낀다, 2: 많이 느끼는 편이다, 3: 조금 느끼는 편이다, 4: 거의 느끼지 않는다
df['BP1'].unique()

array([ 4.,  3.,  1.,  2.,  0., nan])

In [28]:
# (변경) 기존0-->NaN, 0: 약한스트레스(3~4) 1: 많은스트레스(1~2)
df['BP1'].replace(
    {0:np.nan, 3:0, 4:0, 2:1}, inplace=True)
df['BP1'].unique()

array([ 0.,  1., nan])

### BP5

In [29]:
# 1: 예 2: 아니오 3:???
df['BP5'].unique()

array([2, 1, 3])

In [30]:
df['BP5'].replace({2:0, 3:np.nan}, inplace=True)
df['BP5'].unique()

array([ 0.,  1., nan])

### D_1_1

In [31]:
# 1: 매우좋음 2: 좋음 3: 보통 4: 나쁨 5: 매우나쁨 7: ????
df['D_1_1'].unique()

array([3, 4, 2, 5, 1, 7])

In [32]:
# (변경) 0: 나쁨 (4~5), 1: 보통(3), 2:좋음(1~2), 기존 7-->NaN
df['D_1_1'].replace(
    {4:0, 5:0, 3:1, 1:2, 7:np.nan}, inplace=True)
df['D_1_1'].unique()

array([ 1.,  0.,  2., nan])

### N_DUSUAL

In [33]:
# 1: 많이 섭취(식욕많음) 2: 평소와 비슷 3: 적게 섭취(식욕낮음)
df['N_DUSUAL'].unique()

array([ 1.,  3.,  2., nan])

In [34]:
# (변경) 0: 식욕 낮음 1: 보통 2: 식욕많음
df['N_DUSUAL'].replace({3:0, 2:1, 1:2}, inplace=True)
df['N_DUSUAL'].unique()

array([ 2.,  0.,  1., nan])

### BH1_1

In [35]:
# 0: 아니오 1: 예 2:????
df['BH1_1'].unique()

array([ 1.,  0.,  2., nan])

In [36]:
# (변경) 기존2-->NaN
df['BH1_1'].replace(2, np.nan, inplace=True)
df['BH1_1'].unique()

array([ 1.,  0., nan])

### BO2_1

In [37]:
# 1: 체중감소노력 2: 체중유지노력 3: 체중증가노력 4: 노력x 5: ???
df['BO2_1'].unique()

array([ 4.,  2.,  1.,  3.,  5., nan])

In [38]:
# (변경) 0: 노력 x 1: 감소노력 2: 유지노력 3: 증가노력 기존5-->NaN
df['BO2_1'].replace({4:0, 5:np.nan}, inplace=True)
df['BO2_1'].unique()

array([ 0.,  2.,  1.,  3., nan])

### BO1_1

In [39]:
# 1: 변화없음 2: 체중감소 3: 체중증가 4: ????
df['BO1_1'].unique()

array([ 1.,  2.,  3.,  4., nan])

In [40]:
# (변경) 0: 없음 1: 감소 2: 증가 기존4-->NaN
df['BO1_1'].replace(4, np.nan, inplace=True)
df['BO1_1'] = df['BO1_1']-1
df['BO1_1'].unique()

array([ 0.,  1.,  2., nan])

### DI1_pt

In [41]:
# 0: 아니오 1: 예 2:?????
df['DI1_pt'].unique()

array([ 0.,  1.,  2., nan])

In [42]:
# (변경) 기존2-->NaN
df['DI1_pt'].replace(2, np.nan, inplace=True)
df['DI1_pt'].unique()

array([ 0.,  1., nan])

### DI1_2

In [43]:
# 1: 매일 복용 2: 20일 이상 복용 3: 15일 이상 복용 4: 복용하지 않음 5: 15일 미만 복용 6: 비해당 7: 모름, 무응답
df['DI1_2'].unique()

array([ 6.,  1.,  4.,  7.,  5.,  2., nan])

In [44]:
# (변경) 7-->NaN, 0 (복용x, 비해당), 1 (15일 미만복용), 2(15일이상 복용), 3(20일 이상 복용), 4(매일 복용)
df['DI1_2'].replace(
    {7:np.nan, 1:4, 2:3, 3:2, 4:0, 5:1, 6:0}, inplace=True)
df['DI1_2'].unique()

array([ 0.,  4., nan,  1.,  3.])

### 심한금단증상경험여부

In [45]:
# 1: 예 2: 아니오 0: ????
df['심한금단증상경험여부'].unique()

array([ 2.,  1.,  0., nan])

In [46]:
# (변경) 기존0-->NaN, 0: 아니오 1: 예
df['심한금단증상경험여부'].replace({0:np.nan, 2:0}, inplace=True)
df['심한금단증상경험여부'].unique()

array([ 0.,  1., nan])

### 전문가의금연권유

In [47]:
# 1: 전문가권유 2: 전문가 상담 3: 약복용(껌, 패치 포함), 4: 없음
df['전문가의금연권유'].unique()

array([ 4.,  1.,  3.,  2., nan])

In [48]:
# (변경) 0: 없음, 1: 있음(1~3)
df['전문가의금연권유'].replace({4:0, 2:1, 3:1}, inplace=True)
df['전문가의금연권유'].unique()

array([ 0.,  1., nan])

In [49]:
df.to_csv('./survey.csv', index=False)